Task 3

In [10]:
import numpy as np
import pandas as pd
import cv2

import matplotlib
#matplotlib.use('TkAgg')
#%matplotlib
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import scipy
import scipy.io as sio
from sklearn.metrics import mean_squared_error 
from scipy.spatial.distance import cdist
import random 

In [2]:
def recoverTransformation(img1, img2, corr1, corr2):
    #Image1 and Image2 are original and transformed input image
    #respectively
    #MSEPix is mean squared error of pixels of T_image and Image2
    #MSECorPts is mean squared error of correspondence points and
    #transformed points
    #T is recovered transformation
    #T_Image is transformed Image
    
    n = len(corr1)

    #plt.imshow(img1)
    #p1 = plt.ginput(n, timeout=30)

    #plt.imshow(img2)
    #p2 = plt.ginput(n, timeout=30)
    #plt.close()
    
    p1 = corr1.copy()
    
    p2 = corr2.copy()
    
    one = np.ones([n, 1])

    p2 = np.append(p2, one, axis=1)

    A = np.zeros([2*n, 6])

    B = np.zeros([2*n, 1])
    
    k = 0
    l = 0
    for i in range(len(A)):
        for j in range(3):
            if (l == 0):
                A[i][j] = p2[k][j]
            else:
                A[i][j+3] = p2[k][j]
        l = l + 1
        if (l == 2):
            k = k + 1
            l = 0

    l = 0
    for i in range(n):
        B[l] = p1[i][0]
        B[l + 1] = p1[i][1]

        l = l + 2
    
    T = (np.linalg.pinv((np.transpose(A)).dot(A))).dot((np.transpose(A)).dot(B))

    T = np.reshape(T, (2, 3))
    
    height, width = img1.shape[:2]
    
    width = width*2

    T_Image = cv2.warpAffine(img2, T, (width, height))
    
    #plt.imshow(T_Image)
    #p3 = plt.ginput(n, timeout=30)
    #plt.close()
    
    #MSEPix = (np.square(img1 - T_Image)).mean()
    
    #MSECorPts = mean_squared_error(p1, p3) 
    
    return T_Image, T#, MSEPix, MSECorPts

In [3]:
def registerImages(ImageNew, ImageOld, corr1, corr2):
    #ImageNew and ImageOld are input images
    #registerdImage is the output image
    
    #img_t, T, MSEPix, MSECorPts = recoverTransformation(ImageNew, ImageOld, 6)
    
    img_t, T = recoverTransformation(ImageNew, ImageOld, corr1, corr2)
    
    pers = np.zeros([1, 3])

    pers[0][2] = 1
    
    T = np.append(T, pers, axis = 0)
    
    mask = np.ones([ImageOld.shape[0], ImageOld.shape[1]])*255
    
    height, width = ImageNew.shape[:2]
    
    width = width*2
    
    img_t = cv2.warpPerspective(ImageOld, T, (width, height))

    T_mask = cv2.warpPerspective(mask, T, (width, height))
    
    registeredImage = ImageNew.copy()
    
    extra_space = np.zeros(ImageNew.shape)
    
    registeredImage = np.append(registeredImage, extra_space, axis = 1)
    
    width_count = 0
    
    width_check = 0
    
    for y in range(T_mask.shape[0]):
        for x in range(T_mask.shape[1]):
            if(T_mask[y][x] == 255):
                if(x in np.arange(width/2)):
                    width_count = width_count + 1
                    width_check = 1
        if(width_check == 1):
            break
        
    mul_percent = 1
    
    mul_percent_change = 1 / width_count
    
    for y in range(T_mask.shape[0]):
        for x in range(T_mask.shape[1]):
            if(T_mask[y][x] == 255):
                if(x in np.arange(width/2)):
                    registeredImage[y][x] = registeredImage[y][x]*mul_percent + img_t[y][x]*(1 - mul_percent)
                    mul_percent = mul_percent - mul_percent_change
                else:
                    registeredImage[y][x] = img_t[y][x]
        mul_percent = 1
                
    #1- get correspondence
    #2- recover transformation (use code of part 1)
    #3- Transform ImageOld to a new matrix of ImageNew size – use
       #cv2.warpAffine() for affine and cv2.warpPerspective() for
       #perspective transformation.
    #4- Generate a mask of 1's and 0's (1's will represent where pixels
       #of ImageOld will land and 0's will represent where pixels of
       #ImageNew will land)

    return registeredImage

In [117]:
def stiching(image1, image2, grey):
    
    set_no = 'set4_'
    
    height, width = image1.shape[:2]

    if(grey == 0):
        image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    else:
        image1_gray = image1

    image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    combine = np.append(image1, image2, axis=1)

    sift = cv2.xfeatures2d.SIFT_create()

    [f1, d1] = sift.detectAndCompute(image1_gray, None)

    [f2, d2] = sift.detectAndCompute(image2_gray, None)
    
    draw =  np.zeros(image1.shape)
    
    draw = cv2.drawKeypoints(image1, f1, draw)
    
    cv2.imwrite(set_no + 'featurePoints.jpg', draw)
    
    f1_mat = np.zeros([4, len(d1)])
    
    f2_mat = np.zeros([4, len(d2)])
    
    for i in range(len(d1)):
        f1_mat[0][i], f1_mat[1][i] = f1[0].pt
        f1_mat[2][i] = f1[0].size
        f1_mat[3][i] = f1[0].angle
    
    for i in range(len(d2)):
        f2_mat[0][i], f2_mat[1][i] = f2[0].pt
        f2_mat[2][i] = f2[0].size
        f2_mat[3][i] = f2[0].angle
    
    sio.savemat('f1_set4.mat', {"f1":f1_mat})
    
    sio.savemat('f2_set4.mat', {"f2":f2_mat})

    sio.savemat('d1_set4.mat', {"d1":d1})
    
    sio.savemat('d2_set4.mat', {"d2":d2})

    eucl_dist = cdist(d1, d2, metric='euclidean')

    #print(d1.shape)
    #print(d2.shape)
    #print(eucl_dist.shape)

    eucl_dist_flatten = eucl_dist.flatten()

    min_idx = np.argsort(eucl_dist_flatten)
    #print(min_idx.shape)

    descrip_num = 300

    feature1 = np.zeros([descrip_num, 2])

    feature2 = np.zeros([descrip_num, 2])

    for i in range(descrip_num):

            f1_in = (round(f1[int(min_idx[i] / eucl_dist.shape[1])].pt[0]), round(f1[int(min_idx[i] / eucl_dist.shape[1])].pt[1]))
            f2_in = (round(f2[int(min_idx[i] % eucl_dist.shape[1])].pt[0] + width), round(f2[int(min_idx[i] % eucl_dist.shape[1])].pt[1]))
            f2_in_in = (round(f2[int(min_idx[i] % eucl_dist.shape[1])].pt[0]), round(f2[int(min_idx[i] % eucl_dist.shape[1])].pt[1]))

            feature1[i] = f1_in

            feature2[i] = f2_in_in

            #combine = cv2.circle(combine, f1_in, round(f1[int(min_idx[i] / eucl_dist.shape[1])].size), color = (0, 0, 255), thickness = 2)
            #combine = cv2.circle(combine, f2_in, round(f2[int(min_idx[i] % eucl_dist.shape[1])].size), color = (0, 255, 0), thickness = 2) 

            combine = cv2.circle(combine, f1_in, 10, color = (0, 0, 255), thickness = 2)
            combine = cv2.circle(combine, f2_in, 10, color = (0, 255, 0), thickness = 2) 


            combine = cv2.line(combine, f1_in, f2_in, (0, 255, 255)) 

    cv2.imwrite(set_no + 'matches.jpg', combine)

    corr_num = 3

    corres1 = np.zeros([corr_num, 2])

    corres2 = np.zeros([corr_num, 2])

    numofiter = 1000

    idx_check = 0

    check = 0

    prevhist = 0

    height, width = image1.shape[:2]

    while(numofiter > 0):

        idx1 = random.sample(range(1, len(feature1)), corr_num)

        for i in range(corr_num):
            corres1[i] = feature1[idx1[i]]
            corres2[i] = feature2[idx1[i]]   

        img_t, T = recoverTransformation(image1, image2, corres1, corres2)

        one = np.ones([len(feature2), 1])

        feature2_append = np.append(feature2, one, axis=1)

        feature2_append = np.transpose(feature2_append)

        key_t = T.dot(feature2_append)

        key_t_reshape = np.reshape(key_t, (len(feature2), 2)) 

        key_t_reshape2 = key_t_reshape.copy()

        for i in range(len(key_t_reshape)):
            key_t_reshape2[i][0] = key_t[0][i]
            key_t_reshape2[i][1] = key_t[1][i]


        if(check == 1):
            prevhist = hist

        hist = 0

        thresh = 3

        for i in range(len(feature1)):
            ECorPts = np.sqrt( (feature1[i][0] - key_t_reshape2[i][0])**2 + (feature1[i][1] - key_t_reshape2[i][1])**2 )
            if(ECorPts < thresh):
                hist = hist + 1

        if (hist > prevhist):
            end_key_t = key_t_reshape2.copy()
        else:
            hist = prevhist

        check = 1

        numofiter = numofiter - 1


    inliers = np.zeros([len(feature1), 1])

    feature1_append = np.append(feature1, inliers, axis=1)

    feature1_inliers_len = 0

    for i in range(len(feature1)):
        ECorPts = np.sqrt( (feature1[i][0] - end_key_t[i][0])**2 + (feature1[i][1] - end_key_t[i][1])**2 )
        if(ECorPts < thresh):
            feature1_append[i][2] = 1 
            feature1_inliers_len = feature1_inliers_len + 1

    #print(feature1_inliers_len)

    final_feature1 = np.zeros([feature1_inliers_len, 2])

    final_feature2 = np.zeros([feature1_inliers_len, 2])

    k = 0

    for i in range(len(feature1)):
        if(feature1_append[i][2] == 1):
            final_feature1[k] = feature1[i]
            final_feature2[k] = feature2[i]

            k = k + 1     

    image_t = registerImages(image1, image2, final_feature1, final_feature2)

    combine = np.append(image1, image2, axis=1)

    for i in range(len(final_feature1)):
        fe1_in = int(final_feature1[i][0]), int(final_feature1[i][1])
        fe2_in = int(final_feature2[i][0] + width), int(final_feature2[i][1])

        combine = cv2.circle(combine, fe1_in, 10, color = (0, 0, 255), thickness = 2)
        combine = cv2.circle(combine, fe2_in, 10, color = (0, 255, 0), thickness = 2)

        combine = cv2.line(combine, fe1_in, fe2_in, (0, 255, 255))
        
    cv2.imwrite(set_no + 'bestMatches.jpg', combine)
        
    return image_t



In [118]:
path_task3_set1 = '/Users/Adam/Desktop/Assignment 5/task3/Set1/'

path_task3_set2 = '/Users/Adam/Desktop/Assignment 5/task3/Set2/'

path_task3_set3 = '/Users/Adam/Desktop/Assignment 5/task3/Set3/'

path_task3_set4 = '/Users/Adam/Desktop/Assignment 5/task3/Set4/'

build1 = cv2.imread(path_task3_set4 + 'image1.jpg')

build2 = cv2.imread(path_task3_set4 + 'image2.jpg')

#build1 = cv2.imread(path_task3_set3 + 'image3.jpg')

#build2 = cv2.imread(path_task3_set3 + 'image4.jpg')

In [119]:
build_t = stiching(build1.copy(), build2.copy(), 0)

#build_t = stiching(build_t.copy(), build3.copy(), 1)

cv2.imwrite('set3_panorama.jpg', build_t)

print("done")   

done
